# Libraries, organizing your code

The classic HDL way is to write a simple file with one `@block` decorated hardware unit and import it as a module. However this is not sufficient in some cases when full portability or extended configureability is desired.

Also, one might use a library of existing, external VHDL or Verilog code.

## User libraries

When starting a new set of hardware elements from scratch, the best approach is to create `@block_component` objects inside a module class.

In particular, when a hardware description should simulate and synthesize, it is desirable to have a base class and use inheritance to implement more
features or architecture-specific variants.

* [Library concepts: Library classes](../notebooks/library_concepts.ipynb#Library-classes)


## External HDL libraries


* [External black boxes](../notebooks/blackboxes.ipynb#External-implementations): Wrapping external HDL files for cyrite usage